In [1]:
from xbbg import blp
import pdblp
import workdays
import datetime
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", 70)

In [2]:
def ME(df):
    for i in range(1,len(df)):
        if df.loc[i,"Date"].month != df.loc[i-1,"Date"].month:
            if df.loc[i-1,"Date"].month ==3:
                df.loc[i-1,"ME"] = 3
            else:
                df.loc[i-1,"ME"] = 1
    df.loc[len(df)-1,"ME"] = 3
    
    return df
  

L = ["ES","NQ","TP"]
T = [i+"1 Index" for i in L]
    
d_from = "20070301"
d_to = workdays.workday(datetime.datetime.today(), days=-1).strftime("%Y%m%d")

df = blp.bdh(T, ["px_last"], d_from, d_to, Calendar="5D", Fill="P" ).reset_index()
df = df[["index"]+T]
df.columns = ["Date"] + L
df.insert(1,"ME",0)

df = ME(df)
df

,Date,ME,ES,NQ,TP
0,2007-03-01,0,1321.12,1930.28,1227.3
1,2007-03-02,0,1303.01,1895.69,1211.1
2,2007-03-05,0,1290.32,1881.96,1173.3
3,2007-03-06,0,1312.18,1916.28,1193.8
4,2007-03-07,0,1310.07,1908.04,1193.1
...,...,...,...,...,...
4317,2023-09-18,0,4501.50,15414.50,2405.5
4318,2023-09-19,0,4490.00,15375.25,2412.5
4319,2023-09-20,0,4447.00,15152.00,2389.0
4320,2023-09-21,0,4372.00,14862.75,2363.0


In [4]:
def Ret(k):
    tmp = df[df["ME"] >= k].copy()
    tmp[L] = tmp[L].pct_change()
    tmp = tmp.dropna().reset_index(drop=True)[["Date"] + L]
    return tmp
    # tmp = df.loc[df["ME"]>=k,["Date"]+[x for x in L]]
    # for j in L:
    #     tmp[j] = tmp[j].pct_change()
    # tmp = tmp.iloc[1:].reset_index(drop=True)
    # tmp = tmp[["Date"]+L]
    

def Percent(tmp):       
    tmp[L] = (tmp[L] * 100).round(1).astype(str) + '%'   
    return tmp
    # for i in range(len(tmp)):
    #     tmp.iloc[i,1:] = tmp.iloc[i,1:].apply("{:.1%}".format)

tmp = Percent(Ret(3))
print("Yearly Ret",tmp.shape)
tmp

Yearly Ret (17, 4)


,Date,ES,NQ,TP
0,2008-03-31,-10.0%,-3.8%,-28.1%
1,2009-03-31,-39.8%,-31.5%,-34.3%
2,2010-03-31,49.1%,58.7%,27.1%
3,2011-03-31,15.2%,19.9%,-9.6%
4,2012-03-30,8.1%,18.6%,1.3%
5,2013-03-29,13.3%,3.0%,23.8%
6,2014-03-31,21.1%,28.5%,16.9%
7,2015-03-31,12.2%,21.5%,29.9%
8,2016-03-31,1.3%,4.1%,-11.2%
9,2017-03-31,16.3%,21.8%,14.9%


In [5]:
tmp = Percent(Ret(1))
print("Monthly Ret",tmp.shape)
tmp

Monthly Ret (198, 4)


,Date,ES,NQ,TP
0,2007-04-30,4.0%,4.9%,-1.0%
1,2007-05-31,3.0%,2.8%,3.4%
2,2007-06-29,-2.1%,-0.1%,0.9%
3,2007-07-31,-3.5%,-0.5%,-3.9%
4,2007-08-31,1.0%,2.5%,-5.6%
...,...,...,...,...
193,2023-05-31,0.0%,7.4%,3.3%
194,2023-06-30,6.0%,5.9%,7.8%
195,2023-07-31,2.8%,3.4%,1.7%
196,2023-08-31,-2.1%,-2.0%,0.2%


In [6]:
def SD(k):
    tmp = df[df["ME"] >= k][L].pct_change().dropna().reset_index(drop=True)   
    tmp = np.std(tmp) * 12**0.5
    SD = pd.DataFrame({"Name": L, "SD": tmp}).sort_values("SD", ascending=False).reset_index(drop=True)
    return SD

# def SD(k):
#     tmp = df.loc[df["ME"]>=k,["Date"]+[x  for x in L]]
#     for j in L:
#         tmp[j] =  tmp[j].pct_change()
#     tmp = tmp.iloc[1:].reset_index(drop=True)
#     tmp = tmp[L]
#     tmp = np.std(tmp) * 12**0.5

#     SD = pd.DataFrame(np.zeros([len(tmp),2]),columns=["Name","SD"])
#     SD["Name"] = L
#     SD["SD"] = tmp.values
#     SD = SD.sort_values("SD", ascending=False).reset_index(drop=True)
#     return SD


tmp = SD(3)
print("SD",tmp.shape)
tmp["SD"] = tmp["SD"].apply("{:.2%}".format)
tmp

SD (3, 2)


,Name,SD
0,NQ,79.13%
1,ES,73.74%
2,TP,69.49%


In [7]:
def Corr(k):
    tmp = df[df["ME"] >= k][L].pct_change().dropna().reset_index(drop=True)
    tmp = tmp.corr()
    tmp = tmp.applymap(lambda x: "{:.2f}".format(x)).replace("1.00", "-")    
    return tmp

# def Corr(k):
#     tmp = df.loc[df["ME"]>=k,["Date"]+[ x for x in L]]
#     for j in L:
#         tmp[j] =  tmp[j].pct_change()

#     tmp = tmp.iloc[1:].reset_index(drop=True)
#     tmp = tmp[L]
#     tmp = tmp[1:].corr()

#     for i in range(len(tmp)):
#         tmp.iloc[i,:] = tmp.iloc[i,:].apply("{:.2f}".format)
    
#     tmp = tmp.replace("1.00", "-")
#     return tmp

tmp = Corr(1)
print("Correlation")
tmp

Correlation


,ES,NQ,TP
ES,-,0.92,0.66
NQ,0.92,-,0.62
TP,0.66,0.62,-


In [8]:
def R(k,SD):
    tmp = Ret(3).iloc[[k],1:].T
    tmp1 =  ["Name","Ret","SD","Ratio"]

    YR = pd.DataFrame(np.zeros([len(tmp),len(tmp1)]),columns=tmp1)
    YR["Name"] = L
    YR["Ret"] = tmp.values
    YR = YR.sort_values("Ret", ascending=False).reset_index(drop=True)

    for i in range(len(YR)):
        YR.loc[i,"SD"] = SD.loc[SD["Name"]==YR.loc[i,"Name"],"SD"].values

    YR["Ratio"] = YR["Ret"] / YR["SD"]

    for j in ["Ret","SD"]:
        YR[j] = YR[j].apply("{:.1%}".format)

    for j in ["Ratio"]:
        YR[j] = YR[j].apply("{:.2f}".format)

    return YR

SD_M = SD(3)
Ret_Y = Ret(3)
for i in range(len(Ret_Y)):    
    exec(f"YR{i}=R(i,SD_M)")

r = 3
c1 = [date.strftime("%Y%m") if (idx % 2 == 0) else " " for idx, date in enumerate(Ret_Y["Date"].tolist() * 2)]

Rank = pd.DataFrame( np.zeros([r,len(c1)]) )
c2 = ["Name","Ret"] * int(len(c1)/2)
c = pd.MultiIndex.from_arrays([c1, c2 ])
Rank.columns = c

Rank2 = pd.DataFrame( np.zeros([r,len(c1)]) )
c2 = ["Name","Ratio"] * int(len(c1)/2)
c = pd.MultiIndex.from_arrays([c1, c2 ])
Rank2.columns = c

Worst = Rank.copy()
Worst2 = Rank2.copy()

for j in range(len(Ret_Y)):
    exec(f"tmp = YR{j}")
    Rank.iloc[:,0+2*j:2+2*j] = tmp.loc[:r-1,['Name','Ret']].values
    
    tmp = tmp.sort_values("Ratio", ascending=False).reset_index(drop=True)
    Rank2.iloc[:,0+2*j:2+2*j] = tmp.loc[:r-1,['Name','Ratio']].values
    
    tmp = tmp.sort_values("Ratio", ascending=True).reset_index(drop=True)
    Worst2.iloc[:,0+2*j:2+2*j] = tmp.loc[:r-1,["Name","Ratio"]].values
    
    a = list(tmp["Ret"])
    for i in range(len(a)):   
        b = a[i]
        a[i] = float(b[:len(b)-1])/100
    tmp["Ret"] = a
    tmp = tmp.sort_values("Ret", ascending=True).reset_index(drop=True)
    tmp["Ret"] = tmp["Ret"].apply("{:.1%}".format)
    
    Worst.iloc[:,0+2*j:2+2*j] = tmp.loc[:r-1,["Name","Ret"]].values

Rank

,200803,,201003,,201203,,201403,,201603,,201803,,202003,,202203,,202309,,200903,,201103,,201303,,201503,,201703,,201903,,202103,,202303,
,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret
0,NQ,-3.8%,NQ,-31.5%,NQ,58.7%,NQ,19.9%,NQ,18.6%,TP,23.8%,NQ,28.5%,TP,29.9%,NQ,4.1%,NQ,21.8%,NQ,20.1%,NQ,10.6%,NQ,4.4%,NQ,68.7%,ES,15.1%,TP,5.8%,TP,19.0%
1,ES,-10.0%,TP,-34.3%,ES,49.1%,ES,15.2%,ES,8.1%,ES,13.3%,ES,21.1%,NQ,21.5%,ES,1.3%,ES,16.3%,TP,15.9%,ES,6.7%,TP,-9.2%,ES,56.1%,NQ,13.7%,ES,-10.7%,NQ,9.0%
2,TP,-28.1%,ES,-39.8%,TP,27.1%,TP,-9.6%,TP,1.3%,NQ,3.0%,TP,16.9%,ES,12.2%,TP,-11.2%,TP,14.9%,ES,11.9%,TP,-4.9%,ES,-9.3%,TP,42.5%,TP,2.2%,NQ,-13.1%,ES,3.2%


In [9]:
Worst

,200803,,201003,,201203,,201403,,201603,,201803,,202003,,202203,,202309,,200903,,201103,,201303,,201503,,201703,,201903,,202103,,202303,
,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret,Name,Ret
0,TP,-28.1%,ES,-39.8%,TP,27.1%,TP,-9.6%,TP,1.3%,NQ,3.0%,TP,16.9%,ES,12.2%,TP,-11.2%,TP,14.9%,ES,11.9%,TP,-4.9%,ES,-9.3%,TP,42.5%,TP,2.2%,NQ,-13.1%,ES,3.2%
1,ES,-10.0%,TP,-34.3%,ES,49.1%,ES,15.2%,ES,8.1%,ES,13.3%,ES,21.1%,NQ,21.5%,ES,1.3%,ES,16.3%,TP,15.9%,ES,6.7%,TP,-9.2%,ES,56.1%,NQ,13.7%,ES,-10.7%,NQ,9.0%
2,NQ,-3.8%,NQ,-31.5%,NQ,58.7%,NQ,19.9%,NQ,18.6%,TP,23.8%,NQ,28.5%,TP,29.9%,NQ,4.1%,NQ,21.8%,NQ,20.1%,NQ,10.6%,NQ,4.4%,NQ,68.7%,ES,15.1%,TP,5.8%,TP,19.0%


In [10]:
Rank2

,200803,,201003,,201203,,201403,,201603,,201803,,202003,,202203,,202309,,200903,,201103,,201303,,201503,,201703,,201903,,202103,,202303,
,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio
0,TP,-0.40,ES,-0.54,NQ,0.74,NQ,0.25,NQ,0.24,TP,0.34,NQ,0.36,TP,0.43,NQ,0.05,NQ,0.28,NQ,0.25,NQ,0.13,NQ,0.06,NQ,0.87,ES,0.20,TP,0.08,TP,0.27
1,ES,-0.14,TP,-0.49,ES,0.67,ES,0.21,ES,0.11,ES,0.18,ES,0.29,NQ,0.27,ES,0.02,ES,0.22,TP,0.23,ES,0.09,TP,-0.13,ES,0.76,NQ,0.17,NQ,-0.17,NQ,0.11
2,NQ,-0.05,NQ,-0.40,TP,0.39,TP,-0.14,TP,0.02,NQ,0.04,TP,0.24,ES,0.17,TP,-0.16,TP,0.21,ES,0.16,TP,-0.07,ES,-0.13,TP,0.61,TP,0.03,ES,-0.14,ES,0.04


In [11]:
Worst2

,200803,,201003,,201203,,201403,,201603,,201803,,202003,,202203,,202309,,200903,,201103,,201303,,201503,,201703,,201903,,202103,,202303,
,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio,Name,Ratio
0,NQ,-0.05,NQ,-0.40,TP,0.39,TP,-0.14,TP,0.02,NQ,0.04,TP,0.24,ES,0.17,TP,-0.16,TP,0.21,ES,0.16,TP,-0.07,TP,-0.13,TP,0.61,TP,0.03,ES,-0.14,ES,0.04
1,ES,-0.14,TP,-0.49,ES,0.67,ES,0.21,ES,0.11,ES,0.18,ES,0.29,NQ,0.27,ES,0.02,ES,0.22,TP,0.23,ES,0.09,ES,-0.13,ES,0.76,NQ,0.17,NQ,-0.17,NQ,0.11
2,TP,-0.40,ES,-0.54,NQ,0.74,NQ,0.25,NQ,0.24,TP,0.34,NQ,0.36,TP,0.43,NQ,0.05,NQ,0.28,NQ,0.25,NQ,0.13,NQ,0.06,NQ,0.87,ES,0.20,TP,0.08,TP,0.27
